Aim to complete as much of this tutorial on your own *before* coming to the practical session.

Use the practical session to get help for any aspect you do not understand or were unable to complete.

# Clustering 1

Learning objectives
1. Apply k-Means, HCA, and kNN using the popular python library [sklearn](https://scikit-learn.org/stable/)
2. Interpret the results to learn about the data structure 
3. Visualise the clustering results 

## Import specific packages and functions
You should have `numpy`, `pandas`, `matplotlib` and `sklearn` already installed from the previous sessions.

In [ ]:
import pandas as pd ## pandas creates data frames/ data tables similar to an excel file, although most python functions like the fromat of a numpy array
import numpy as np ### for working with numpy arrays

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn import metrics
from sklearn.metrics import silhouette_score, silhouette_samples

from scipy.cluster.hierarchy import dendrogram

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from matplotlib.colors import colorConverter
#%matplotlib inline

## Load in dataset
In this tutorial we will aim to predict the occurence of COVID-19 using the same data used in BIDS 2.

In [ ]:
# Use pandas to load the data

# Load the data
#df = pd.read_csv('../Data-main/COVID19_proteomics.xlsx', index_col=0) ## path to file, may not need the "index_col=0" paramater depending on operating system
df = pd.read_excel("../Data-main/COVID19_proteomics.xlsx")
# View the first 5 records to see what features we have in our dataset should start with first column called 'COVID19'
df.head()

As you can see this data set in its current format is not suitable for our algorithms. 
In python column indices start from 0, we want to select only the proteomics columns for feature scaling.

We want to predict COVID19 (column 0 ) using columns 3:453

As we can see, the data is now in the proper format for our KNN models.

Now that out data has been pre-processed, we can seperate the target from the features.

In [ ]:
# Create feature matrix and target vector
X = df.iloc[:,3:]
y = df['COVID19']

In order to test our alogrithms we need to set aside some of the data we have. This is practice for machine learning models. We will use 80% of our data to train our model, and the remaining 20% will be used to test the performance of our model. 

[sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) has a function ```train_test_split``` to easily do this for us.

In [ ]:
# enter your CID here, or date of birth, or another number of your choosing to use as random state
CID = 0

# remember to check the documentation of each algorithm if setting the random_state is needed
# for this tutorial (where it is clearly being used) and all upcoming tutorials...

In [ ]:
# Split the df into 80% train 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20, random_state=CID)

### Normalise Data
Our data has been cleaned and split into training and testing features and targets. We have one last pre-processing step to take before being able to run our data: scaling. As we mentioned before, these methods (k-Means, kNN, etc.) make predictions based on distances between data points. As such, it is crucial that all of the data it is comparing is on the same scale. In our proteomics data, most of the data is continuous. We will scale the data using the ```StandardScaler()``` shown in the previous tutorials. 

When scaling your data you want to fit the model to your training data, and only transform your testing data. 

In [ ]:
# Instantiate scaler model
scaler = StandardScaler()

# Fit and Transform X_train
X_train = scaler.fit_transform(X_train)

# Transform X_test
X_test = scaler.transform(X_test)

## K-Means Clustering

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(X.iloc[:,0], X.iloc[:,1]) ### plotting first two proteomics features
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=CID) ### kmeans clustering 
kmeans.fit(X) ## fit on X and define the random state

## How do we pick the number of clusters? 
K-means clustering requires that the number of clusters are predefined, however we would want justify how many clusters are *optimal*. We can treat ```n_clusters``` as a hyperparamater and for loop through several number of clusters. To select the best number of clusters we will implement the elbow method, where we train multiple models using a different number of clusters and storing the value of the ```intertia_``` property, the Within Cluster Sum of Squares (WCSS), every time.

WCSS is defined as the sum of the squared distance between each member of the cluster and its centroid.

In [ ]:
wcss = [] ### create a list 

for i in range(2, 12): ## loop through n_clusters 2:11
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=CID)
    kmeans.fit(X_train) ## fit on X and define the random state
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(2, 12), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

How many clusters would you pick? We've chosen 4 below, but please change it to what you think is right.

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=CID)
pred_y = kmeans.fit_predict(X_train)
plt.scatter(X_train[:,0], X_train[:,1], c=pred_y, s=50, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0],kmeans.cluster_centers_[:, 1], s=300, c='red')
plt.show()

It looks like there are 4 clusters at the shapest elbow point. However, when plotting just the first two proteomics features it is not clear and their is overlap of the clusters.
However, we can use alternative methods to visualise these clusters. One of these is PCA from the BIDS2 tutorial.

In [ ]:
pca = PCA(n_components=4)
pca_covid = pca.fit_transform(X_train)
# run PCA with 4 components
# plot a scatterplot using seaborn
# the x axis will contain the first column of the pca scores x=pca_covid[:, 0]
plt.scatter(pca_covid[:, 0], pca_covid[:, 1], c=pred_y, s=50, cmap='viridis')
plt.show()

You can also do this with other dimension reduction techniques such as MDS, t-SNE, UMAP, etc. (see tutorials and optional materials from BIDS 2-4 for code to reuse here).

For example for MDS:

In [ ]:
embedding = MDS(n_components=4, metric=True, random_state=CID)
embedding_covid = embedding.fit_transform(X_train)
plt.scatter(embedding_covid[:, 0], embedding_covid[:, 1], c=pred_y, s=50, cmap='viridis')
plt.show()

Now, is there a way to plot all 4 components from the MDS instead of the first 2...?

In [ ]:
## Reuse code from a previous tutorial here



## Hierarchical Clustering (or Hierarchical Cluster Analysis, HCA)

This is similar to K-means although this time we wil use the ```AgglomerativeClustering``` function

In [ ]:
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')
cluster.fit(X_train)
labels = cluster.fit_predict(X_train)

In [ ]:
silhouette_score(X_train,labels,metric="euclidean",sample_size=1000,random_state=CID)

To mix things up we'll use a different colourmap ([cmap](https://matplotlib.org/stable/gallery/color/colormap_reference.html)), have a look at different ones in [Matplotlib](https://matplotlib.org/stable/) and see which one you prefer?

In [ ]:
## visualise as a scatter plot with the PCA components
plt.scatter(pca_covid[:, 0], pca_covid[:, 1], c=cluster.labels_, s=50, cmap='rainbow')
plt.show()

### Like in the K-means approach, defining the *optimal* number of clusters requires iterating over ```n_clusters```. This time we will implement the silhouette method.

The same challenge occours, where we need to define the ```n_clusters```. This time we will use the silhouette score to detemine optimal number of clusters. The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The value of the silhouette ranges between [1, -1], where a high value indicates that the object is well matched to its own cluster and poorly matched to neighbouring clusters. If most objects have a high value, then the clustering configuration is appropriate. If many points have a low or negative value, then the clustering configuration may have too many or too few clusters.

In [ ]:
## Silhoutte plot function no need to change this it's a custom function to obtain silhouete plots
def silhouette_plot(X, y, n_clusters):

    # Compute the silhouette scores for each sample
    silhouette_avg = silhouette_score(X, y)
    sample_silhouette_values = silhouette_samples(X, y)

    y_lower = padding = 2
    for i in range(n_clusters):
        ax = plt.gca()
        # Aggregate the silhouette scores for samples belonging to
        ith_cluster_silhouette_values = sample_silhouette_values[y == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        cmap = cm.get_cmap("Spectral")
        color = cmap(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                         0,
                         ith_cluster_silhouette_values,
                         facecolor=color,
                         edgecolor=color,
                         alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i + 1))

        # Compute the new y_lower for next plot
        y_lower = y_upper + padding
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhoutte score of all the values
    ax.axvline(x=silhouette_avg, c='r', alpha=0.8, lw=0.8, ls='-')
    ax.annotate('Average',
                xytext=(silhouette_avg, y_lower * 1.025),
                xy=(0, 0),
                ha='center',
                alpha=0.8,
                c='r')

    ax.set_yticks([])  # Clear the yaxis labels / ticks
    ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_ylim(0, y_upper + 1)
    ax.set_xlim(-0.075, 1.0)
    plt.figure()
    return ax

In [ ]:
silho = [] 
for i in range(2,11):
    # Run the HCC algorithm
    cluster = AgglomerativeClustering(n_clusters=i, affinity='euclidean', linkage='ward')
    cluster.fit(X_train)
    labels = cluster.fit_predict(X_train)
    silhouette_plot(X_train, labels, n_clusters=i)
    silho.append(silhouette_score(X_train,labels,metric="euclidean",sample_size=1000,random_state=CID))
plt.plot(range(2, 11), silho)
plt.title('Silhouette Method')
plt.xlabel('Number of clusters')
plt.ylabel('Avg. Silhouette Coeficient')
plt.show()    

Here we can see that the optimal K (the one with the highest average Silhoutte coeficient) is 2. However, often averages can be deceiving (e.g. outliers with either very low coeficients or extremely high coeficients can drastically change the average). With the silhouette method it is a good idea to examine the coeficients of each sample in each cluster independently as well.

As we did with the k-means approach we can also visualise these clusters, as a scatter plot, using various techniques. 

In [ ]:
### edit this to show the two clusters
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
cluster.fit(X_train)
labels = cluster.fit_predict(X_train)

# plot a scatterplot using seaborn
# the x axis will contain the first column of the pca scores x=pca_covid[:, 0]
plt.scatter(pca_covid[:, 0], pca_covid[:, 1], c=labels, s=50, cmap='viridis')
plt.show()

One of the most well-known methods of visualisation for hierarchical clustering is by means of a [dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html).

In [ ]:
### this helper function is required, no need to edit this, you may need to also install scipy if this is not installed yet
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(X)
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)

Why does the plot below look different? What is the difference between `level` and `lastp`?

In [ ]:
# plot the top three clusters of the dendrogram
plot_dendrogram(model, truncate_mode="lastp", p=3)

Are there any outliers?

## k-Nearest Neighbours (kNN)

### Scikit-Learn's kNN 


Let's apply the [kNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) algorithm to a Covid-19 proteomics data set

### Theory Recap (Optional)
k-Nearest Neighbours is a simple machine learning model that makes predictions based off of the most similar observations in its traing data. While it can be very powerful, its predictive capability is limited to observations that are similar to what the training data it has in memory.

Unlike most other models, kNN does not 'learn' from its training dataset. Instead it holds the entire training set in memory and then compares the new observation to its stored data. kNN performs no work until a prediction is required.

When a prediction is required it does exactly what it name says. The model examines the new observation and finds the most similar records (nearest neighbours) that it holds in its training set. The number of neighbours (k) the model selects from its training data is defined by the user. 

A prediction can be made by either returning the most common outcome (classification) or by taking the average (regression).

**Some Important Notes on Using kNN**

- kNN is a simple model to implement, but as a result it is limited in the types of data it can take as input.
- When working with kNN the phrase "rubbish in, rubbish out" is never more accurate.
- kNN does not handle categorical variables so everything must be pre-processed to include numerical values only.
- Additionally, as you will see in the next section, the nearest neighbours are found by calculating the distances between the new observation and the records held in memory. Those with the smallest distances are considered most similar.
- Intuitively, you should understand the importance of scaling your data (so that they are all being measured on the same metric) before running a kNN model. 

If you have kept with us this far, it is about to pay off, it's time to run some models!

### Baseline Accuracy

When evaluating model performance we want to start with a baseline accuracy. This is the accuracy score if we were to simply guess the majority outcome everytime. It gives us a starting point to compare our models to. The baseline metric is the best we can do without models. Hopefully, our models can improve over the baseline.

In [ ]:
# Calculate the baseline accuracy

# Find the majority count
y_train.value_counts()

In [ ]:
# COVID counts for y_test
y_test.value_counts()

The above numbers depend on the `random_state` of the `train_test_split` function. So you may see a difference compared to your _neighbour_ (pardon the pun).

Change the cell below to the number from your test set - which group has more samples?

In [ ]:
# If we were to guess the majority (1 or 0) for each test sample, we would get a total of ... correct
# The baseline accuracy is the number of correct guesses divided by total guesses 
baseline = np.max(y_test.value_counts()) / np.sum(y_test.value_counts())
baseline

### COVID predictions using a kNN classifier

Let's start by making predictions and calculating the accuracy of Scikit-Learn's [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) model.

This can be done by the following steps:

- Instantiate the model
- Fit the model with our training data
- Make predictions based off of our test features
- Provide the known test targets to determine the accuracy

In [ ]:
# Instantiate the KNClassifier object
scikit_KNN = KNeighborsClassifier(n_neighbors=9)

# Fit the model with training data
scikit_KNN.fit(X_train, y_train)

# Make predictions
print(scikit_KNN.predict(X_test))

# Calculate accuracy
scikit_KNN.score(X_test, y_test)

In [ ]:
## Accuracy is one performance metric here we will define several alternative metrics 

def modelPerformance(confMat):
    TN = confMat[0, 0]
    TP = confMat[1, 1]
    FP = confMat[0, 1]
    FN = confMat[1, 0]
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    spec = TN / (TN + FP)
    fpr = FP / (TN + FP)
    f1 = 2 * (prec * rec) / (prec + rec)
    acc = (TP + TN) / (TP + FP + TN + FN)
    return (acc, prec, rec, spec, fpr, f1)

def printPerformance(confMat):
    acc, prec, rec, spec, fpr, f1 = modelPerformance(confMat)
    print("Accuracy = " "%.4f" % acc)
    print("Precision = " "%.4f" % prec)
    print("Recall = " "%.4f" % rec)
    print("Specificity = " "%.4f" % spec)
    print("False positive rate = " "%.4f" % fpr)
    print("F1-score = " "%.4f" % f1)
    np.set_printoptions(precision=2)
    print("Confusion matrix (%):")
    print(confMat/np.sum(confMat)*100)

In [ ]:
y_test_hat = scikit_KNN.predict(X_test)
cmat = metrics.confusion_matrix(y_test, y_test_hat)
printPerformance(cmat)

Complete the sentence below with your results:

Considering the baseline accuracy was ...%, sklearn's model is an improvement/reduction at ...%

### What's next?
We have gone through how to implement a kNN model that will work with either classification or regression problems. kNN is included here as it assignes a cluster (class or, more broadly, a _value_) based on its similarity to other samples in the neighbourhood (which can be considered a cluster). Then we walked through a classification problem using the COVID-19 proteomics dataset.

For further understanding and practice:
- Plot kNN results like above 
- Cluster a different data set using HCA and k-means
- Use a different dataset for a regression problem, look up: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
- Implement different distance metrics such as the Manhattan distance or Hamming distance and see how they change results

## Your turn
Select another dataset from the `Data` folder, import and inspect it.

Are the same parameters as used above optimal for the new dataset? Do different parameters changes the outcome?

Can you identify some potential outliers with these methods?

In [ ]:
# Import datasets...

In [ ]:
# Perform scaling (if needed)

In [ ]:
# Applying some of the clustering methods

In [ ]:
# Visualise your results using some of the dimension reduction techniques (for k-Means and kNN) or via a dendrogram (HCA)